Universidade Federal do Rio Grande do Sul (UFRGS)   
Programa de Pós-Graduação em Engenharia Civil (PPGEC)   

# PEC00144: Experimental Methods in Civil Engineering

### Reading the serial port of an Arduino device  

---
_Prof. Marcelo M. Rocha, Dr.techn._ [(ORCID)](https://orcid.org/0000-0001-5640-1020)  
_Porto Alegre, RS, Brazil_ 


In [14]:
# Importing Python modules required for this notebook
# (this cell must be executed with "shift+enter" before any other Python cell)

import sys
import time
import serial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from   MRPy  import MRPy


### 1. Setup serial communication

In order to run this notebook, the Python module ``pyserial`` must be installed.
To ensure the module availability, open a conda terminal and issue the command:

    conda install -c anaconda pyserial

Before openning the serial port, verify with Arduino IDE which USB identifier the 
board has be assigned (in Windows it has the form "COMxx", while in Linux it
it is something like "/dev/ttyXXXX").


In [15]:
#port = '/dev/ttyUSB0' 
#baud =  9600

port  = 'COM5'            # change this address according to your computer
baud  =  115200           # match this number with the Arduino's output baud rate

Ardn  =  serial.Serial(port, baud, timeout=1)

time.sleep(3)             # this is important to give time for serial settling


### 2. Define function for reading one incoming line

In [16]:
def ReadSerial(nchar, nvar, nlines=1):

    Ardn.write(str(nlines).encode())
    data = np.zeros((nlines,nvar))

    for k in range(nlines):

        wait = True
        while(wait):
          
            if (Ardn.inWaiting() >= nchar):
                wait = False
                
                bdat = Ardn.readline() 
                sdat = bdat.decode()
                sdat = sdat.replace('\n',' ').split()
            
                data[k, :] = np.array(sdat[0:nvar], dtype='int')
        
    return data


### 3. Acquire data lines from serial port

In [17]:
try:
    data = ReadSerial(30, 4, nlines=4096)
    t    = data[:,0 ]
    acc  = data[:,1:]

    Ardn.close()
    print('Acquisition ok!')
    
except:
    Ardn.close()
    sys.exit('Acquisition failure!')


Acquisition ok!


### 4. Create ``MRPy`` instance and save to file 

In [18]:
ti  = (t - t[0])/1000
a   =  2*9.81*acc/2**15
 
data = MRPy.resampling(ti, a).zero_mean()
data.to_file('read_MPU6050', form='excel')

print('Average sampling rate is {0:5.1f}Hz.'.format(data.fs))

print(data.mean(axis=1))


Average sampling rate is 367.8Hz.
[-2.77555756e-17  2.77555756e-17  4.02455846e-16]


### 5. Data visualization

In [ ]:
fig1 = data.plot_time(fig=1, figsize=(12,8), axis_t=[0, data.Td, -20, 20])


Trabalho para 10/02/2021:

Fazer uma aquisição com o MPU6050 preso com dupla face numa régua em balanço e apresentar resultado no tempo e na frequência. Qual a frequência natural no modo fundamental da régua.
